# Key Performance Indicators of Business Insights


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Upload and copy the main dataset
flat_dataset = pd.read_csv('eda_kpi.csv')
df_kpi = flat_dataset.copy()

In [ ]:
df_kpi.shape

(39218, 14)

# Before dive into what and Why we need KPI. Lets recall what is core business problem

> # Core Business Problem**
The static discounting model fails to adapt to changes in customer demand, competitor pricing, or product lifecycle. The business lacks a system to learn from real-time user behaviour such as views, carts, and purchases. They often overshoot discounts, killing margins, or undershoot them, missing conversion opportunities.

> ## GOAL:
- Product performance
- Discount effectiveness
- Revenue leakage
- Customer behavior trends
- Strategic opportunities for pricing or inventory adjustments


In [ ]:
df_kpi.head(10)

,order_id,order_date,user_id,sku_id,category,brand,quantity,price_per_unit,discount_applied,revenue,profit,days_since_launch,viewed_same_day,pricing_error_flag
0,O000001,2023-08-27,U4418,P1477,Apparel,BrandC,2,794.70,30.0,1589.40,-312.60,-83.0,0,1
1,O000002,2024-08-06,U3995,P0935,Sports,BrandC,5,1912.46,20.0,9562.30,8229.00,72.0,0,0
2,O000003,2024-11-29,U5880,P1126,Electronics,BrandC,2,621.70,0.0,1243.40,210.78,803.0,0,0
3,O000004,2025-07-03,U1969,P1491,Electronics,BrandA,6,1679.62,0.0,10077.72,5141.46,1113.0,0,0
4,O000005,2024-04-20,U1925,P0274,Beauty,BrandD,2,658.59,20.0,1317.18,599.80,203.0,0,0
5,O000006,2024-07-16,U1615,P1356,Electronics,BrandD,1,1169.55,40.0,1169.55,653.24,1098.0,0,0
6,O000007,2025-03-02,U4146,P0911,Books,BrandD,1,1212.28,40.0,1212.28,215.37,231.0,0,0
7,O000008,2023-08-21,U1608,P0447,Books,BrandD,1,787.49,30.0,787.49,-149.16,267.0,0,1
8,O000009,2023-11-05,U5490,P0903,Sports,BrandF,1,1006.28,20.0,1006.28,432.26,755.0,0,0
9,O000010,2025-05-02,U1789,P1395,Sports,BrandA,4,1154.54,30.0,4618.16,3590.24,1474.0,0,0


In [ ]:
print(df_kpi['order_id'].nunique())
print(df_kpi['sku_id'].count())

39218
39218


In [ ]:
#filtering out the rows without "UNKNOWN"
df_kpi['sku_known_flag'] = df_kpi['sku_id'] != 'UNKNOWN'

In [ ]:
eda_kpi_filtered = df_kpi[df_kpi['sku_known_flag']]


In [ ]:
unknown_summary = df_kpi[~df_kpi['sku_known_flag']].agg({
    'revenue': 'sum',
    'profit': 'sum',
    'order_id': 'count'
})


In [ ]:
print(unknown_summary)

revenue     894217.42
profit           0.00
order_id       363.00
dtype: float64


In [ ]:
df_kpi['sku_known_flag'] = df_kpi['sku_id'] != 'UNKNOWN'


In [ ]:
df_kpi_known = df_kpi[df_kpi['sku_known_flag']]


In [ ]:
unknown_summary = df_kpi[~df_kpi['sku_known_flag']].agg({
    'revenue': 'sum',
    'profit': 'sum',
    'order_id': 'count'
}).rename({
    'revenue': 'Unknown Revenue',
    'profit': 'Unknown Profit',
    'order_id': 'Unknown Orders'
})

In [ ]:
print(unknown_summary)

Unknown Revenue    894217.42
Unknown Profit          0.00
Unknown Orders        363.00
dtype: float64


***8.94L revenue and 363 orders are untraceable to product-level insights due to missing SKU mappings. These records have zero recorded profit and cannot be used in profitability or product performance KPIs.***

------------------------------------------------------------------------------------------

In [ ]:
# Filtering  out rows where sku_id is NOT 'UNKNOWN'
df_kpi_known = df_kpi[df_kpi['sku_id'] != 'UNKNOWN'].copy()  #a version of your flat KPI dataset without that one problematic 'UNKNOWN' row,
# which was skewing the top revenue and zero-profit analysis.

# **KPI 1 : TOP PERFORMING SKU**

In [ ]:
# LETS DIVE IN WITH OUR KPI 1
# Top Performing SKUs (by Revenue & Profit)
top_skus = df_kpi_known.groupby('sku_id').agg(
    total_revenue=('revenue', 'sum'),
    total_profit=('profit', 'sum'),
    total_orders=('order_id', 'nunique')
).sort_values('total_revenue', ascending=False).head(10)

top_skus

,total_revenue,total_profit,total_orders
sku_id,,,
P0535,135608.840,78087.780,40
P0404,118647.660,84570.210,40
P0068,115039.475,35381.715,35
P1174,107686.000,55852.080,38
P0249,107418.190,77153.950,43
P0878,106734.710,88788.280,29
P0946,105654.320,61559.720,36
P0299,105410.420,59430.420,30
P0224,105358.740,63597.270,39


In [ ]:
df_kpi_known['revenue'].min()

180.05

In [ ]:
print(df_kpi_known.shape)
print(df_kpi_known['sku_id'].nunique())
print(df_kpi_known['revenue'].min())


(38855, 16)
1500
180.05


________________________________________________________________________________________________________________________________________________________________


# **KPI 2 : BOTTOM PERFORMING SKU**

In [ ]:
# Aggregation without query to inspect bottom SKUs
# KPI 2
low_skus_check = df_kpi_known.groupby('sku_id').agg(
    total_revenue=('revenue', 'sum'),
    total_orders=('order_id', 'nunique')
).sort_values('total_revenue')

low_skus_check.head(10)  # This should show your lowest revenue SKUs


,total_revenue,total_orders
sku_id,,
P0067,22142.01,16
P0932,22775.27,11
P1173,23713.16,13
P0894,24315.96,16
P0681,24794.92,17
P0328,25604.70,16
P1131,26048.22,12
P0657,27945.22,16
P0582,28057.39,21


In [ ]:
# Creating clean KPI base by removing 'UNKNOWN' SKU
df_kpi_kn = df_kpi[df_kpi['sku_id'] != 'UNKNOWN'].copy()

In [ ]:
print("Cleaned KPI dataset shape:", df_kpi_kn.shape)
print(" Remaining unique SKUs:", df_kpi_kn['sku_id'].nunique())
print(" Sample rows:")
df_kpi_kn.head(3)

Cleaned KPI dataset shape: (38855, 14)
 Remaining unique SKUs: 1500
 Sample rows:


,order_id,order_date,user_id,sku_id,category,brand,quantity,price_per_unit,discount_applied,revenue,profit,days_since_launch,viewed_same_day,pricing_error_flag
0,O000001,2023-08-27,U4418,P1477,Apparel,BrandC,2,794.70,30.0,1589.4,-312.60,-83.0,0,1
1,O000002,2024-08-06,U3995,P0935,Sports,BrandC,5,1912.46,20.0,9562.3,8229.00,72.0,0,0
2,O000003,2024-11-29,U5880,P1126,Electronics,BrandC,2,621.70,0.0,1243.4,210.78,803.0,0,0


________________________________________________________________________________________________________________________________________________________________

# **KPI 3: Revenue by Category/Brand**

In [ ]:
#We want to calculate total revenue grouped by Brand AND Category
#category and revenue
rev_by_cat = df_kpi_kn.groupby('category')['revenue'].sum().reset_index().sort_values('revenue', ascending=False)

#brand by revenue
rev_by_brand = df_kpi_kn.groupby('brand')['revenue'].sum().reset_index().sort_values(by='revenue', ascending=False)

# with both
rev_brdcat = df_kpi_kn.groupby(['brand', 'category'])['revenue'].sum().reset_index().sort_values('revenue', ascending=False)


In [ ]:
rev_by_cat.head(6)

,category,revenue
1,Beauty,1.715668e+07
3,Electronics,1.615428e+07
5,Sports,1.568913e+07
4,Home & Kitchen,1.536051e+07
0,Apparel,1.525084e+07
2,Books,1.518265e+07


## Deep dive Insight:
1. Beauty leads in total revenue – signaling a strong demand or successful pricing strategy.

2. All top 6 categories are performing fairly close, which means assortment strategy is diversified and balanced.

3. Books, despite being a traditionally lower-ticket category, are almost as strong as Electronics – interesting point to validate further

In [ ]:
rev_by_brand.head(6)

,brand,revenue
5,BrandF,1.662123e+07
2,BrandC,1.473882e+07
1,BrandB,1.393703e+07
4,BrandE,1.366236e+07
0,BrandA,1.323245e+07
3,BrandD,1.319358e+07


## Insights:

1. BrandF is a top performer and likely offers a wide category presence.

2. Brands C–E are closely competing → Potential candidates for pricing/discount strategy comparisons.

3. Consider evaluating BrandF’s margin, discount behavior, and repeat purchase rates.



In [ ]:
rev_brdcat.head(6)

,brand,category,revenue
31,BrandF,Beauty,3096882.220
33,BrandF,Electronics,2994631.580
35,BrandF,Sports,2888893.055
34,BrandF,Home & Kitchen,2876749.680
2,BrandA,Books,2816010.325
13,BrandC,Beauty,2795859.520


## Insight:

1. BrandF is dominating in multiple categories, making it a power brand across verticals – deserves deeper analysis on its pricing structure.

2. BrandA in Books is punching above its weight → niche bestseller strategy or price-optimization working?

3. Beauty + BrandC is another important intersection – maybe they are using higher discounts?



----------------------------------------------------------------------------------------------------------------------------------------------------------------

# **KPI 4: Average Discount by Brand/Category**

I want to compute the mean of discount_applied for:

Brand

Category

Combined Category–Brand

In [ ]:
df_kpi_kn.head(3)

,order_id,order_date,user_id,sku_id,category,brand,quantity,price_per_unit,discount_applied,revenue,profit,days_since_launch,viewed_same_day,pricing_error_flag,sku_known_flag
0,O000001,2023-08-27,U4418,P1477,Apparel,BrandC,2,794.70,30.0,1589.4,-312.60,-83.0,0,1,True
1,O000002,2024-08-06,U3995,P0935,Sports,BrandC,5,1912.46,20.0,9562.3,8229.00,72.0,0,0,True
2,O000003,2024-11-29,U5880,P1126,Electronics,BrandC,2,621.70,0.0,1243.4,210.78,803.0,0,0,True


In [ ]:
# discount for brand
dis_brd = df_kpi_kn.groupby('brand')['discount_applied'].mean().reset_index().sort_values('discount_applied', ascending=False)

#discount for category
dis_cat = df_kpi_kn.groupby('category')['discount_applied'].mean().reset_index().sort_values('discount_applied', ascending=False)

#discou nt on brand and categories
dis_brdcat = df_kpi_kn.groupby(['brand', 'category'])['discount_applied'].mean().reset_index().sort_values('discount_applied', ascending=False)

In [ ]:
dis_brd.head(6)

,brand,discount_applied
4,BrandE,17.828924
5,BrandF,17.651414
2,BrandC,17.570156
0,BrandA,17.540471
3,BrandD,17.489459
1,BrandB,17.470068


In [ ]:
dis_cat.head(6)

,category,discount_applied
3,Electronics,17.739329
1,Beauty,17.665870
5,Sports,17.608863
4,Home & Kitchen,17.526146
2,Books,17.496787
0,Apparel,17.389084


In [ ]:
dis_brdcat.head(6)

,brand,category,discount_applied
28,BrandE,Home & Kitchen,18.135755
15,BrandC,Electronics,18.113402
35,BrandF,Sports,18.104167
31,BrandF,Beauty,18.042122
7,BrandB,Beauty,17.985989
3,BrandA,Electronics,17.979858


## NOW, Is this growth organic or fueled by aggressive discounting?


**Visible** ->
category: Beauty,
revenue: ₹17.16M,
Discount:	17.67%, Insight: Healthy growth with moderate discounting

# Category Summary:

1. No category is discounting aggressively (>20%), which indicates the platform isn’t heavily dependent on markdowns.

2. Apparel is the star, achieving ₹15.25M revenue with the lowest discount (17.39%).

3. Beauty and Electronics are both efficient earners, not overspending on promotions.

**Visible**
Brand: BrandF, 	Revenue: ₹16.62M, Avg. Discount: 17.65%, Insight: Top revenue brand with controlled discount -> power brand

## Brand Summary:

1. BrandF is your flagship: highest revenue with average discounts, indicating demand-based pricing is working.

2. BrandE may require a margin check – highest discount in top group.

3. BrandB and BrandD are your most efficient: stable discounts and solid revenue.

-----------------------------------------------------------------------------------------------------------------------------------------------

# **KPI 5: Profit Margin % by Brand and Category**

In [ ]:
df_kpi_kn.head()

,order_id,order_date,user_id,sku_id,category,brand,quantity,price_per_unit,discount_applied,revenue,profit,days_since_launch,viewed_same_day,pricing_error_flag,sku_known_flag
0,O000001,2023-08-27,U4418,P1477,Apparel,BrandC,2,794.70,30.0,1589.40,-312.60,-83.0,0,1,True
1,O000002,2024-08-06,U3995,P0935,Sports,BrandC,5,1912.46,20.0,9562.30,8229.00,72.0,0,0,True
2,O000003,2024-11-29,U5880,P1126,Electronics,BrandC,2,621.70,0.0,1243.40,210.78,803.0,0,0,True
3,O000004,2025-07-03,U1969,P1491,Electronics,BrandA,6,1679.62,0.0,10077.72,5141.46,1113.0,0,0,True
4,O000005,2024-04-20,U1925,P0274,Beauty,BrandD,2,658.59,20.0,1317.18,599.80,203.0,0,0,True


In [ ]:
#The core formula i going to use is ->  Profit Margin (%) = (Profit / Revenue) * 100 by brand
profit_margin_brand = df_kpi_kn.groupby('brand').agg({
    'revenue': 'sum',
    'profit': 'sum'
}).assign(profit_margin_pct=lambda x: (x['profit'] / x['revenue']) * 100).reset_index().sort_values(by='profit_margin_pct', ascending=False)



In [ ]:
# Profit Margin (%) = (Profit / Revenue) * 100 by category
profit_margin_cat = df_kpi_kn.groupby('category').agg({
    'profit':'sum',
    'revenue': 'sum'
}).assign(profit_margin_pct = lambda x: (x['profit'] / x['revenue']) * 100).reset_index().sort_values(by='profit_margin_pct', ascending=False)

In [ ]:
profit_margin_brand.head(6)

,brand,revenue,profit,profit_margin_pct
0,BrandA,1.323245e+07,7168757.85,54.175588
5,BrandF,1.662123e+07,8859874.00,53.304569
6,No Brand,9.408615e+06,4995900.87,53.099215
1,BrandB,1.393703e+07,7388522.23,53.013597
2,BrandC,1.473882e+07,7783491.60,52.809470
3,BrandD,1.319358e+07,6674934.80,50.592290


In [ ]:
profit_margin_cat.head(6)

,category,profit,revenue,profit_margin_pct
0,Apparel,8353669.760,1.525084e+07,54.775154
4,Home & Kitchen,8317294.610,1.536051e+07,54.147267
1,Beauty,9102343.165,1.715668e+07,53.054226
3,Electronics,8549336.435,1.615428e+07,52.923032
2,Books,7587026.360,1.518265e+07,49.971684
5,Sports,7838835.825,1.568913e+07,49.963497


## Strategic Diagnosis:
1. High Revenue + High Margin → True Winners
- BrandF and BrandA are the two standout brands:
- BrandF = Powerhouse in revenue (₹16.6M) and solid margin (53.3%)

2. BrandA = Less revenue but highest efficiency at 54.18% and lowest avg. discount -> ideal candidate for price expansion or premium positioning

3. “No Brand” is a Wildcard
Generating ₹9.4M with 53% margin -> if these are unbranded/private-label products, they are doing extremely well without promotional support.

4. BrandD needs a watch
Margins slightly lower at 50.6% compared to others above 52%.




## Business recommendation

- BrandA, BrandF:  **Lock margin thresholds** in pricing engine; Avoid over-discounting            |
- BrandD : **Investigate cost drivers**, consider nudging pricing slightly higher       
- No Brand: Explore more aggressively – could build into house brand or pricing experiments


# **KPI 6: Orders per SKU per Month (Velocity )**
How frequently a SKU is being sold per month, useful for inventory & pricing alignment.

In [ ]:
# df_kpi_kn['order_date'].dtype
# df_kpi_kn['order_date'] = pd.to_datetime(df_kpi_kn['order_date'])
df_kpi_kn['year_month'] = df_kpi_kn['order_date'].dt.to_period('M')

In [ ]:
# sku per month : the quantity of a specific Stock Keeping Unit (SKU) sold within a given month.

sku_per_month = df_kpi_kn.groupby(['sku_id', 'year_month']).size().reset_index(name='order_per_month')

In [ ]:
# what is the average oder per month per sku
#how frequently a specific product (SKU) is picked or sold within a given timeframe, often within a warehouse or retail setting
sku_velocity = sku_per_month.groupby('sku_id')['order_per_month'].mean().reset_index().rename(columns={'order_per_month': 'avg_monthly_orders'}).sort_values(by='avg_monthly_orders', ascending=False)


In [ ]:
print("-------------Top Fast-Moving SKUs by Monthly Order Volume-------------------\n")
sku_velocity.head(6)

-------------Top Fast-Moving SKUs by Monthly Order Volume-------------------



,sku_id,avg_monthly_orders
103,P0104,2.583333
797,P0798,2.384615
1343,P1344,2.357143
239,P0240,2.312500
1179,P1180,2.285714
176,P0177,2.250000


## KPI: SKU Velocity – Orders per Month


| Rank | SKU ID | Avg. Monthly Orders | Insight                                                               |
| ---- | ------ | ------------------- | --------------------------------------------------------------------- |
| 1️  | P0104  | **2.58**            |  Fastest moving SKU – high potential for stable pricing or bundling |
| 2️  | P0798  | 2.38                |  High demand – good candidate for inventory alignment               |
| 3️  | P1344  | 2.36                |  Consistent performer – likely a steady seller                      |
| 4️  | P0240  | 2.31                |  Watch for seasonality vs steady demand                             |
| 5️  | P1180  | 2.29                |  Add to dynamic pricing candidate list                               |
| 6️  | P0177  | 2.25                |  Stable monthly flow – test price elasticity here                   |


# RECOMMENDATION

  SKU ID	  ->             Action
- P0104	->  Lock margin + stabilize price, consider raising if elasticity is low
- P0798	-> Include in dashboard for weekly velocity tracking
- P0177	 -> Candidate for small dynamic price variation testing
- All 6	-> Add velocity_score into your RL dataset for policy gradient or Q-table learning.


sku_Velocity is the most important feature for our further works like forecasting and RL engine. So, we need a better and normalize sku_Velocity features.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# normalize it for use in ML models:

sku_velocity['velocity_score'] = MinMaxScaler().fit_transform(sku_velocity[['avg_monthly_orders']])


# **Why Build a velocity_score?**

- Quantify SKU Demand Speed
- Use in Downstream Models
- ore Input for RL Agent Decisions like What price action should I take, given a product's behavior? High velocity -> reward price stability or premium testing
-  Business Reporting Simplification

In [ ]:
sku_velocity.to_csv('sku_velocity.csv', index=False)